In [1]:
import multiprocessing as mp
import shutil
import subprocess
from pathlib import Path

import numpy as np
import pandas as pd
import wfdb
import wfdb.io
from tqdm import tqdm

----
setup variables

In [2]:
mimic_root = Path('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0')

In [3]:
dataset_root = Path('/home/thulio/projects/masters-research/data')

In [4]:
min_seg_time = 5 * 60

In [5]:
relevant_signals = ['II', 'PLETH', 'ABP']

In [6]:
interfering_codes = (
    '33',
    '40',
    '410',
    '411',
    '412',
    '413',
    '414',
    '42',
    '43',
    '44',
    '458',
    '459',
    '785'
)

In [7]:
diabetes_codes = (
    '250'
)

In [8]:
merge_tolerance = pd.Timedelta(hours=8)

In [9]:
samples_per_class = 10

----
create headers table

In [10]:
p_folders = sorted([f for g in mimic_root.iterdir() if g.is_dir() for f in g.iterdir() if f.is_dir()])
p_folders

[PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000020'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000030'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000033'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000052'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000079'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000085'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000107'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000109'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000123'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000124'),
 PosixPath('/home/th

In [11]:
def process_subject_headers(p_folder: Path):
    master_files = [file for file in p_folder.iterdir() if file.name.startswith('p') and not file.name.endswith('n.hea')]
    segment_data = []
    for master_file in master_files:
        try:
            master_header = wfdb.io.rdheader(master_file.as_posix().replace('.hea', ''), rd_segments=True)
        except (FileNotFoundError, TypeError):
            continue
        subject_id_str, record_start_str = master_file.name.split('-', 1)
        subject_id = int(subject_id_str[1:])
        record_start = pd.to_datetime(record_start_str.replace('.hea', ''), format='%Y-%m-%d-%H-%M')
        for seg in master_header.segments:
            seg: wfdb.io.Record | None
            if seg is not None and not seg.record_name.endswith('layout'):
                segment_data.append({
                    'subject_id': subject_id,
                    'record_start': record_start,
                    'segment_id': seg.record_name,
                    'segment_time': seg.sig_len / seg.fs,
                    'segment_signals': seg.sig_name,
                    'master_id': master_file.stem,
                })
    return segment_data


with mp.Pool(10) as p:
    data = list(tqdm(p.imap_unordered(func=process_subject_headers, iterable=p_folders), total=len(p_folders)))
df_waveforms = pd.DataFrame([y for x in data for y in x])
df_waveforms['record_start'] = pd.to_datetime(df_waveforms['record_start'])
df_waveforms

100%|██████████| 10282/10282 [02:27<00:00, 69.62it/s]


,subject_id,record_start,segment_id,segment_time,segment_signals,master_id
0,30,2172-10-16 12:22:00,3524877_0001,3600.000,"[II, V]",p000030-2172-10-16-12-22
1,33,2116-12-25 13:11:00,3737936_0001,3060.000,"[II, MCL1]",p000033-2116-12-25-13-11
2,33,2116-12-24 12:35:00,3713820_0001,120.000,"[II, MCL1]",p000033-2116-12-24-12-35
3,33,2116-12-24 12:35:00,3713820_0002,27036.504,"[II, MCL1]",p000033-2116-12-24-12-35
4,33,2116-12-24 12:35:00,3713820_0003,53603.496,"[II, MCL1]",p000033-2116-12-24-12-35
...,...,...,...,...,...,...
856334,99383,2131-06-12 18:28:00,3912142_0013,16890.000,"[RESP, CVP, ABP, PLETH, II, I, V]",p099383-2131-06-12-18-28
856335,99383,2131-06-12 18:28:00,3912142_0014,28.000,"[RESP, CVP, ABP, II, I, V]",p099383-2131-06-12-18-28
856336,99383,2131-06-12 18:28:00,3912142_0015,20274.000,"[RESP, CVP, ABP, PLETH, II, I, V]",p099383-2131-06-12-18-28
856337,99383,2131-06-12 18:28:00,3912142_0016,15.000,"[RESP, CVP, ABP, II, I, V]",p099383-2131-06-12-18-28


In [12]:
df_waveforms.to_csv('csvs/waveforms.csv', index=False)

----

In [11]:
df_waveforms = pd.read_csv('csvs/waveforms.csv', parse_dates=['record_start'])
df_waveforms

,subject_id,record_start,segment_id,segment_time,segment_signals,master_id
0,30,2172-10-16 12:22:00,3524877_0001,3600.000,"['II', 'V']",p000030-2172-10-16-12-22
1,33,2116-12-25 13:11:00,3737936_0001,3060.000,"['II', 'MCL1']",p000033-2116-12-25-13-11
2,33,2116-12-24 12:35:00,3713820_0001,120.000,"['II', 'MCL1']",p000033-2116-12-24-12-35
3,33,2116-12-24 12:35:00,3713820_0002,27036.504,"['II', 'MCL1']",p000033-2116-12-24-12-35
4,33,2116-12-24 12:35:00,3713820_0003,53603.496,"['II', 'MCL1']",p000033-2116-12-24-12-35
...,...,...,...,...,...,...
856334,99383,2131-06-12 18:28:00,3912142_0013,16890.000,"['RESP', 'CVP', 'ABP', 'PLETH', 'II', 'I', 'V']",p099383-2131-06-12-18-28
856335,99383,2131-06-12 18:28:00,3912142_0014,28.000,"['RESP', 'CVP', 'ABP', 'II', 'I', 'V']",p099383-2131-06-12-18-28
856336,99383,2131-06-12 18:28:00,3912142_0015,20274.000,"['RESP', 'CVP', 'ABP', 'PLETH', 'II', 'I', 'V']",p099383-2131-06-12-18-28
856337,99383,2131-06-12 18:28:00,3912142_0016,15.000,"['RESP', 'CVP', 'ABP', 'II', 'I', 'V']",p099383-2131-06-12-18-28


In [12]:
valid_segment_time = df_waveforms['segment_time'] > min_seg_time
valid_segment_signals = df_waveforms['segment_signals'].apply(lambda sl: all([sig in sl for sig in relevant_signals]))
df_waveforms_filt = df_waveforms[valid_segment_time & valid_segment_signals]
df_waveforms_filt

,subject_id,record_start,segment_id,segment_time,segment_signals,master_id
154,160,2174-11-06 10:12:00,3531764_0003,10789.0,"['RESP', 'PLETH', 'II', 'V', 'AVR', 'ABP']",p000160-2174-11-06-10-12
539,188,2162-01-10 18:53:00,3285727_0007,17396.0,"['PLETH', 'ABP', 'RESP', 'II', 'V', 'AVR']",p000188-2162-01-10-18-53
541,188,2162-01-10 18:53:00,3285727_0009,44047.0,"['PLETH', 'ABP', 'RESP', 'II', 'V', 'AVR']",p000188-2162-01-10-18-53
543,188,2162-01-10 18:53:00,3285727_0011,537.0,"['PLETH', 'ABP', 'RESP', 'II', 'V', 'AVR']",p000188-2162-01-10-18-53
545,188,2162-01-10 18:53:00,3285727_0013,3487.0,"['PLETH', 'ABP', 'RESP', 'II', 'V', 'AVR']",p000188-2162-01-10-18-53
...,...,...,...,...,...,...
856331,99383,2131-06-12 18:28:00,3912142_0010,6007.0,"['RESP', 'ABP', 'PLETH', 'II', 'I', 'V']",p099383-2131-06-12-18-28
856332,99383,2131-06-12 18:28:00,3912142_0011,3463.0,"['RESP', 'CVP', 'ABP', 'PLETH', 'II', 'I', 'V']",p099383-2131-06-12-18-28
856334,99383,2131-06-12 18:28:00,3912142_0013,16890.0,"['RESP', 'CVP', 'ABP', 'PLETH', 'II', 'I', 'V']",p099383-2131-06-12-18-28
856336,99383,2131-06-12 18:28:00,3912142_0015,20274.0,"['RESP', 'CVP', 'ABP', 'PLETH', 'II', 'I', 'V']",p099383-2131-06-12-18-28


----
load mimic tables

In [13]:
df_admissions = pd.read_csv('/home/thulio/Downloads/mimic/ADMISSIONS.csv',
                            index_col='ROW_ID',
                            parse_dates=['ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'EDREGTIME', 'EDOUTTIME']).sort_index()
df_admissions

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
ROW_ID,,,,,,,,,,,,,,,,,,
1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaT,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN,NaT,NaT,NEWBORN,0,1
2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,1
3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,2191-03-15 13:10:00,2191-03-16 01:10:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0,1
4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaT,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,BUDDHIST,NaN,ASIAN,NaT,NaT,NEWBORN,0,1
5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaT,NaT,CHRONIC RENAL FAILURE/SDA,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58972,99985,176670,2181-01-27 02:47:00,2181-02-12 17:05:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,ENGL,JEWISH,MARRIED,WHITE,2181-01-26 23:35:00,2181-01-27 04:18:00,FEVER,0,1
58973,99991,151118,2184-12-24 08:30:00,2185-01-05 12:15:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaT,NaT,DIVERTICULITIS/SDA,0,1
58974,99992,197084,2144-07-25 18:03:00,2144-07-28 17:56:00,NaT,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2144-07-25 13:40:00,2144-07-25 18:50:00,RETROPERITONEAL HEMORRHAGE,0,1


In [14]:
df_icustays = pd.read_csv('/home/thulio/Downloads/mimic/ICUSTAYS.csv',
                          index_col='ROW_ID',
                          parse_dates=['INTIME', 'OUTTIME']).sort_index()
df_icustays

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
ROW_ID,,,,,,,,,,,
1,2,163353,243653,carevue,NICU,NICU,56,56,2138-07-17 21:20:07,2138-07-17 23:32:21,0.0918
2,3,145834,211552,carevue,MICU,MICU,12,12,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646
3,4,185777,294638,carevue,MICU,MICU,52,52,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785
4,5,178980,214757,carevue,NICU,NICU,56,56,2103-02-02 06:04:24,2103-02-02 08:06:00,0.0844
5,6,107064,228232,carevue,SICU,SICU,33,33,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729
...,...,...,...,...,...,...,...,...,...,...,...
61528,99985,176670,279638,metavision,MICU,MICU,52,52,2181-01-29 05:33:34,2181-02-09 12:45:20,11.2998
61529,99991,151118,226241,metavision,TSICU,TSICU,14,14,2184-12-28 17:30:58,2184-12-31 20:56:20,3.1426
61530,99992,197084,242052,metavision,MICU,MICU,50,50,2144-07-25 18:04:42,2144-07-27 17:27:55,1.9745


In [15]:
df_diagnoses = pd.read_csv('/home/thulio/Downloads/mimic/DIAGNOSES_ICD.csv', index_col='ROW_ID').dropna().sort_index()
df_diagnoses

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
ROW_ID,,,,
1,2,163353,1.0,V3001
2,2,163353,2.0,V053
3,2,163353,3.0,V290
4,3,145834,1.0,0389
5,3,145834,2.0,78559
...,...,...,...,...
651043,99999,113369,1.0,75612
651044,99999,113369,2.0,7861
651045,99999,113369,3.0,4019


----
final processing

In [16]:
df_diagnoses_extra = df_diagnoses.copy()
df_diagnoses_extra['interf'] = df_diagnoses_extra['ICD9_CODE'].str.startswith(interfering_codes)
df_diagnoses_extra['diab'] = df_diagnoses_extra['ICD9_CODE'].str.startswith(diabetes_codes)
df_diagnoses_agg = df_diagnoses_extra.groupby(['SUBJECT_ID', 'HADM_ID'])[['interf', 'diab']].any()
df_diagnoses_more = (~df_diagnoses_agg.groupby('SUBJECT_ID')[['diab']].any()).rename(columns={'diab': 'control'})
df_diagnoses_agg = pd.merge(df_diagnoses_agg, df_diagnoses_more, how='left', left_index=True, right_index=True)
# control = ~df_diagnoses_agg['interf'] & ~df_diagnoses_agg['diab']
control = ~df_diagnoses_agg['interf'] & df_diagnoses_agg['control']
diab = ~df_diagnoses_agg['interf'] & df_diagnoses_agg['diab']
# interf = df_diagnoses_agg['interf']
df_diagnoses_agg['class'] = pd.NA
df_diagnoses_agg['class'] = df_diagnoses_agg['class'].where(~control, 'control')
df_diagnoses_agg['class'] = df_diagnoses_agg['class'].where(~diab, 'diab')
# df_diagnoses_agg['class'] = df_diagnoses_agg['class'].where(~interf, 'interf')
df_diagnoses_agg = df_diagnoses_agg.drop(columns=['interf', 'diab', 'control'])
df_diagnoses_agg = df_diagnoses_agg.dropna().reset_index()
df_diagnoses_agg

,SUBJECT_ID,HADM_ID,class
0,2,163353,control
1,4,185777,control
2,5,178980,control
3,7,118037,control
4,8,159514,control
...,...,...,...
15621,99836,101018,control
15622,99837,197195,control
15623,99928,104802,diab
15624,99934,176121,control


In [17]:
df_wave_icu = pd.merge_asof(df_waveforms_filt.sort_values('record_start'),
                            df_icustays.sort_values('INTIME'),
                            left_on='record_start',
                            right_on='INTIME',
                            left_by='subject_id',
                            right_by='SUBJECT_ID',
                            tolerance=merge_tolerance,
                            direction='backward')
df_wave_icu = df_wave_icu.dropna(subset='SUBJECT_ID')
df_wave_icu

,subject_id,record_start,segment_id,segment_time,segment_signals,master_id,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,76547,2100-09-01 19:49:00,3426661_0036,88395.000,"['II', 'ABP', 'PLETH']",p076547-2100-09-01-19-49,76547.0,171643.0,297858.0,metavision,TSICU,TSICU,14.0,14.0,2100-09-01 17:27:53,2100-09-12 22:10:04,11.1960
1,76547,2100-09-01 19:49:00,3426661_0039,2497.536,"['II', 'ABP', 'PLETH']",p076547-2100-09-01-19-49,76547.0,171643.0,297858.0,metavision,TSICU,TSICU,14.0,14.0,2100-09-01 17:27:53,2100-09-12 22:10:04,11.1960
2,76547,2100-09-01 19:49:00,3426661_0045,14629.000,"['II', 'ABP', 'PLETH']",p076547-2100-09-01-19-49,76547.0,171643.0,297858.0,metavision,TSICU,TSICU,14.0,14.0,2100-09-01 17:27:53,2100-09-12 22:10:04,11.1960
3,76547,2100-09-01 19:49:00,3426661_0034,45936.000,"['II', 'ABP', 'PLETH']",p076547-2100-09-01-19-49,76547.0,171643.0,297858.0,metavision,TSICU,TSICU,14.0,14.0,2100-09-01 17:27:53,2100-09-12 22:10:04,11.1960
4,76547,2100-09-01 19:49:00,3426661_0032,4468.960,"['II', 'ABP', 'PLETH']",p076547-2100-09-01-19-49,76547.0,171643.0,297858.0,metavision,TSICU,TSICU,14.0,14.0,2100-09-01 17:27:53,2100-09-12 22:10:04,11.1960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62873,98185,2208-08-19 14:01:00,3312195_0022,34484.000,"['II', 'ABP', 'PLETH']",p098185-2208-08-19-14-01,98185.0,116667.0,216102.0,metavision,TSICU,TSICU,14.0,14.0,2208-08-19 13:03:37,2208-08-21 05:38:09,1.6906
62874,98185,2208-08-19 14:01:00,3312195_0024,1449.000,"['II', 'ABP', 'PLETH']",p098185-2208-08-19-14-01,98185.0,116667.0,216102.0,metavision,TSICU,TSICU,14.0,14.0,2208-08-19 13:03:37,2208-08-21 05:38:09,1.6906
62875,98185,2208-08-19 14:01:00,3312195_0026,19830.000,"['II', 'ABP', 'PLETH']",p098185-2208-08-19-14-01,98185.0,116667.0,216102.0,metavision,TSICU,TSICU,14.0,14.0,2208-08-19 13:03:37,2208-08-21 05:38:09,1.6906
62876,98185,2208-08-19 14:01:00,3312195_0002,24048.136,"['II', 'ABP', 'PLETH']",p098185-2208-08-19-14-01,98185.0,116667.0,216102.0,metavision,TSICU,TSICU,14.0,14.0,2208-08-19 13:03:37,2208-08-21 05:38:09,1.6906


In [26]:
df_wave_icu_class = pd.merge(df_wave_icu, df_diagnoses_agg, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
df_wave_icu_class

,subject_id,record_start,segment_id,segment_time,segment_signals,master_id,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,class
0,64916,2102-02-21 23:00:00,3010066_0006,2786.0,"['RESP', 'II', 'V', 'AVR', 'PLETH', 'ABP']",p064916-2102-02-21-23-00,64916.0,151439.0,210492.0,metavision,SICU,SICU,33.0,33.0,2102-02-21 21:01:50,2102-02-23 16:23:26,1.8067,control
1,64916,2102-02-21 23:00:00,3010066_0010,17761.0,"['RESP', 'ABP', 'PLETH', 'II', 'V', 'AVR']",p064916-2102-02-21-23-00,64916.0,151439.0,210492.0,metavision,SICU,SICU,33.0,33.0,2102-02-21 21:01:50,2102-02-23 16:23:26,1.8067,control
2,64916,2102-02-21 23:00:00,3010066_0008,36913.0,"['RESP', 'ABP', 'PLETH', 'II', 'V', 'AVR']",p064916-2102-02-21-23-00,64916.0,151439.0,210492.0,metavision,SICU,SICU,33.0,33.0,2102-02-21 21:01:50,2102-02-23 16:23:26,1.8067,control
3,77804,2102-06-17 13:17:00,3598888_0020,2363.0,"['V', 'RESP', 'II', 'AVR', 'PLETH', 'ABP', 'PA...",p077804-2102-06-17-13-17,77804.0,102018.0,228189.0,metavision,SICU,SICU,33.0,33.0,2102-06-17 13:14:50,2102-06-19 18:05:17,2.2017,control
4,77804,2102-06-17 13:17:00,3598888_0011,1145.0,"['V', 'RESP', 'II', 'AVR', 'PLETH', 'ABP', 'PAP']",p077804-2102-06-17-13-17,77804.0,102018.0,228189.0,metavision,SICU,SICU,33.0,33.0,2102-06-17 13:14:50,2102-06-19 18:05:17,2.2017,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1624,75354,2200-11-16 06:29:00,3077393_0007,4284.0,"['II', 'ABP', 'PLETH']",p075354-2200-11-16-06-29,75354.0,100286.0,260324.0,metavision,TSICU,TSICU,14.0,14.0,2200-11-16 00:51:17,2200-11-20 18:06:02,4.7186,control
1625,55030,2201-04-02 22:41:00,3831691_0008,3335.0,"['RESP', 'II', 'PLETH', 'ABP', 'V', 'AVR']",p055030-2201-04-02-22-41,55030.0,187761.0,281141.0,metavision,SICU,SICU,33.0,33.0,2201-04-02 22:23:48,2201-04-06 16:03:49,3.7361,control
1626,55030,2201-04-02 22:41:00,3831691_0011,99454.0,"['RESP', 'II', 'PLETH', 'ABP', 'V', 'AVR']",p055030-2201-04-02-22-41,55030.0,187761.0,281141.0,metavision,SICU,SICU,33.0,33.0,2201-04-02 22:23:48,2201-04-06 16:03:49,3.7361,control
1627,55030,2201-04-02 22:41:00,3831691_0013,42429.0,"['RESP', 'II', 'PLETH', 'ABP', 'V', 'AVR']",p055030-2201-04-02-22-41,55030.0,187761.0,281141.0,metavision,SICU,SICU,33.0,33.0,2201-04-02 22:23:48,2201-04-06 16:03:49,3.7361,control


In [27]:
df_wave_icu_adm_class = pd.merge(df_wave_icu_class, df_admissions, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
df_wave_icu_adm_class

,subject_id,record_start,segment_id,segment_time,segment_signals,master_id,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,...,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,64916,2102-02-21 23:00:00,3010066_0006,2786.0,"['RESP', 'II', 'V', 'AVR', 'PLETH', 'ABP']",p064916-2102-02-21-23-00,64916.0,151439.0,210492.0,metavision,...,Private,SPAN,UNOBTAINABLE,NaN,HISPANIC OR LATINO,2102-02-18 00:48:00,2102-02-18 05:00:00,BRAIN MASS,0,1
1,64916,2102-02-21 23:00:00,3010066_0010,17761.0,"['RESP', 'ABP', 'PLETH', 'II', 'V', 'AVR']",p064916-2102-02-21-23-00,64916.0,151439.0,210492.0,metavision,...,Private,SPAN,UNOBTAINABLE,NaN,HISPANIC OR LATINO,2102-02-18 00:48:00,2102-02-18 05:00:00,BRAIN MASS,0,1
2,64916,2102-02-21 23:00:00,3010066_0008,36913.0,"['RESP', 'ABP', 'PLETH', 'II', 'V', 'AVR']",p064916-2102-02-21-23-00,64916.0,151439.0,210492.0,metavision,...,Private,SPAN,UNOBTAINABLE,NaN,HISPANIC OR LATINO,2102-02-18 00:48:00,2102-02-18 05:00:00,BRAIN MASS,0,1
3,77804,2102-06-17 13:17:00,3598888_0020,2363.0,"['V', 'RESP', 'II', 'AVR', 'PLETH', 'ABP', 'PA...",p077804-2102-06-17-13-17,77804.0,102018.0,228189.0,metavision,...,Medicaid,ENGL,NOT SPECIFIED,SINGLE,WHITE,NaT,NaT,END STAGE LIVER DISEASE,0,1
4,77804,2102-06-17 13:17:00,3598888_0011,1145.0,"['V', 'RESP', 'II', 'AVR', 'PLETH', 'ABP', 'PAP']",p077804-2102-06-17-13-17,77804.0,102018.0,228189.0,metavision,...,Medicaid,ENGL,NOT SPECIFIED,SINGLE,WHITE,NaT,NaT,END STAGE LIVER DISEASE,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1624,75354,2200-11-16 06:29:00,3077393_0007,4284.0,"['II', 'ABP', 'PLETH']",p075354-2200-11-16-06-29,75354.0,100286.0,260324.0,metavision,...,Government,ENGL,NOT SPECIFIED,SINGLE,UNABLE TO OBTAIN,NaT,NaT,RHABDO COMPARTMENT SYNDROME,0,1
1625,55030,2201-04-02 22:41:00,3831691_0008,3335.0,"['RESP', 'II', 'PLETH', 'ABP', 'V', 'AVR']",p055030-2201-04-02-22-41,55030.0,187761.0,281141.0,metavision,...,Private,ENGL,CATHOLIC,MARRIED,WHITE,NaT,NaT,C6-7 TUMOR/SDA,0,1
1626,55030,2201-04-02 22:41:00,3831691_0011,99454.0,"['RESP', 'II', 'PLETH', 'ABP', 'V', 'AVR']",p055030-2201-04-02-22-41,55030.0,187761.0,281141.0,metavision,...,Private,ENGL,CATHOLIC,MARRIED,WHITE,NaT,NaT,C6-7 TUMOR/SDA,0,1
1627,55030,2201-04-02 22:41:00,3831691_0013,42429.0,"['RESP', 'II', 'PLETH', 'ABP', 'V', 'AVR']",p055030-2201-04-02-22-41,55030.0,187761.0,281141.0,metavision,...,Private,ENGL,CATHOLIC,MARRIED,WHITE,NaT,NaT,C6-7 TUMOR/SDA,0,1


In [28]:
elective = df_wave_icu_adm_class['ADMISSION_TYPE'] == 'ELECTIVE'
valid = df_wave_icu_adm_class['class'].isin(['control', 'diab'])
# df_final = df_wave_icu_adm_class[valid & elective]
df_final = df_wave_icu_adm_class[valid]
df_final

,subject_id,record_start,segment_id,segment_time,segment_signals,master_id,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,...,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,64916,2102-02-21 23:00:00,3010066_0006,2786.0,"['RESP', 'II', 'V', 'AVR', 'PLETH', 'ABP']",p064916-2102-02-21-23-00,64916.0,151439.0,210492.0,metavision,...,Private,SPAN,UNOBTAINABLE,NaN,HISPANIC OR LATINO,2102-02-18 00:48:00,2102-02-18 05:00:00,BRAIN MASS,0,1
1,64916,2102-02-21 23:00:00,3010066_0010,17761.0,"['RESP', 'ABP', 'PLETH', 'II', 'V', 'AVR']",p064916-2102-02-21-23-00,64916.0,151439.0,210492.0,metavision,...,Private,SPAN,UNOBTAINABLE,NaN,HISPANIC OR LATINO,2102-02-18 00:48:00,2102-02-18 05:00:00,BRAIN MASS,0,1
2,64916,2102-02-21 23:00:00,3010066_0008,36913.0,"['RESP', 'ABP', 'PLETH', 'II', 'V', 'AVR']",p064916-2102-02-21-23-00,64916.0,151439.0,210492.0,metavision,...,Private,SPAN,UNOBTAINABLE,NaN,HISPANIC OR LATINO,2102-02-18 00:48:00,2102-02-18 05:00:00,BRAIN MASS,0,1
3,77804,2102-06-17 13:17:00,3598888_0020,2363.0,"['V', 'RESP', 'II', 'AVR', 'PLETH', 'ABP', 'PA...",p077804-2102-06-17-13-17,77804.0,102018.0,228189.0,metavision,...,Medicaid,ENGL,NOT SPECIFIED,SINGLE,WHITE,NaT,NaT,END STAGE LIVER DISEASE,0,1
4,77804,2102-06-17 13:17:00,3598888_0011,1145.0,"['V', 'RESP', 'II', 'AVR', 'PLETH', 'ABP', 'PAP']",p077804-2102-06-17-13-17,77804.0,102018.0,228189.0,metavision,...,Medicaid,ENGL,NOT SPECIFIED,SINGLE,WHITE,NaT,NaT,END STAGE LIVER DISEASE,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1624,75354,2200-11-16 06:29:00,3077393_0007,4284.0,"['II', 'ABP', 'PLETH']",p075354-2200-11-16-06-29,75354.0,100286.0,260324.0,metavision,...,Government,ENGL,NOT SPECIFIED,SINGLE,UNABLE TO OBTAIN,NaT,NaT,RHABDO COMPARTMENT SYNDROME,0,1
1625,55030,2201-04-02 22:41:00,3831691_0008,3335.0,"['RESP', 'II', 'PLETH', 'ABP', 'V', 'AVR']",p055030-2201-04-02-22-41,55030.0,187761.0,281141.0,metavision,...,Private,ENGL,CATHOLIC,MARRIED,WHITE,NaT,NaT,C6-7 TUMOR/SDA,0,1
1626,55030,2201-04-02 22:41:00,3831691_0011,99454.0,"['RESP', 'II', 'PLETH', 'ABP', 'V', 'AVR']",p055030-2201-04-02-22-41,55030.0,187761.0,281141.0,metavision,...,Private,ENGL,CATHOLIC,MARRIED,WHITE,NaT,NaT,C6-7 TUMOR/SDA,0,1
1627,55030,2201-04-02 22:41:00,3831691_0013,42429.0,"['RESP', 'II', 'PLETH', 'ABP', 'V', 'AVR']",p055030-2201-04-02-22-41,55030.0,187761.0,281141.0,metavision,...,Private,ENGL,CATHOLIC,MARRIED,WHITE,NaT,NaT,C6-7 TUMOR/SDA,0,1


In [29]:
len(df_final['subject_id'].unique())

181

In [35]:
df_control = df_final[df_final['class'] == 'control'].groupby('subject_id').sample(n=1, random_state=0).sample(n=samples_per_class, random_state=0)
df_diab = df_final[df_final['class'] == 'diab'].groupby('subject_id').sample(n=1, random_state=0).sample(n=samples_per_class, random_state=0)
df_subjects = pd.concat((df_control, df_diab), axis=0)[['subject_id', 'record_start', 'segment_id', 'master_id', 'class']]
df_subjects

,subject_id,record_start,segment_id,master_id,class
838,92381,2150-11-15 22:32:00,3213238_0070,p092381-2150-11-15-22-32,control
396,56038,2125-09-07 22:45:00,3688191_0033,p056038-2125-09-07-22-45,control
740,83263,2145-12-08 23:07:00,3461856_0013,p083263-2145-12-08-23-07,control
78,57752,2109-06-02 21:27:00,3269067_0003,p057752-2109-06-02-21-27,control
386,73471,2125-07-11 14:44:00,3570060_0007,p073471-2125-07-11-14-44,control
802,55044,2150-09-09 16:26:00,3487905_0003,p055044-2150-09-09-16-26,control
204,31574,2114-12-28 09:31:00,3728524_0007,p031574-2114-12-28-09-31,control
208,28775,2116-03-04 18:52:00,3872045_0004,p028775-2116-03-04-18-52,control
1203,67659,2179-07-06 01:15:00,3469659_0027,p067659-2179-07-06-01-15,control
640,85980,2140-09-16 14:33:00,3455932_0013,p085980-2140-09-16-14-33,control


In [23]:
df_subjects.to_csv('csvs/subjects.csv')

----
fetching subjects data

In [33]:
for subject_id, record_start, segment_id, master_id, label in df_subjects.itertuples(index=False):
    pid = f'p{subject_id:06d}'
    original_folder = mimic_root / pid[:3] / pid
    new_folder = dataset_root / label / pid
    shutil.copytree(original_folder, new_folder, dirs_exist_ok=True)
    subprocess.run(['wget', '-N', '-c', '-np', '--show-progress', '-P', new_folder.as_posix(), '-nd', f'https://physionet.org/files/mimic3wdb-matched/1.0/{pid[:3]}/{pid}/{master_id}.hea'])
    subprocess.run(['wget', '-N', '-c', '-np', '--show-progress', '-P', new_folder.as_posix(), '-nd', f'https://physionet.org/files/mimic3wdb-matched/1.0/{pid[:3]}/{pid}/{segment_id}.hea'])
    subprocess.run(['wget', '-N', '-c', '-np', '--show-progress', '-P', new_folder.as_posix(), '-nd', f'https://physionet.org/files/mimic3wdb-matched/1.0/{pid[:3]}/{pid}/{segment_id}.dat'])

--2025-10-17 10:37:10--  https://physionet.org/files/mimic3wdb-matched/1.0/p09/p092381/p092381-2150-11-15-22-32.hea
Resolvendo physionet.org (physionet.org)... 18.18.42.54
Conectando-se a physionet.org (physionet.org)|18.18.42.54|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 304 Not Modified
Arquivo ‘/home/thulio/projects/masters-research/data/control/p092381/p092381-2150-11-15-22-32.hea’ não foi modificado no servidor. Não será baixado.

--2025-10-17 10:37:10--  https://physionet.org/files/mimic3wdb-matched/1.0/p09/p092381/3213238_0070.hea
Resolvendo physionet.org (physionet.org)... 18.18.42.54
Conectando-se a physionet.org (physionet.org)|18.18.42.54|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 304 Not Modified
Arquivo ‘/home/thulio/projects/masters-research/data/control/p092381/3213238_0070.hea’ não foi modificado no servidor. Não será baixado.

--2025-10-17 10:37:11--  https://physionet.org/files/mimic3wdb-matched/1.0/p09/p0923

In [ ]:
for subject_id, record_start, segment_id, _, label in df_subjects.itertuples(index=False):
    pid = f'p{subject_id:06d}'
    original_folder = mimic_root / pid[:3] / pid
    new_folder = dataset_root / label / pid
    shutil.copytree(original_folder, new_folder, dirs_exist_ok=True)
    subprocess.run(['wget', '-r', '-N', '-c', '-np', '--show-progress', '-P', new_folder.as_posix(), '-nd', f'https://physionet.org/files/mimic3wdb-matched/1.0/{pid[:3]}/{pid}/'])